In [10]:
# Import necessary modules
import os
import requests
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import time
from pymysql.err import IntegrityError, OperationalError

In [11]:
# Load environment variables from .env
load_dotenv()

True

In [12]:
# Establish a database connection using SQLAlchemy and credentials from the .env file
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME')

if None in (DB_USER, DB_PASSWORD, DB_HOST, DB_PORT, DB_NAME):
    raise Exception("Database credentials are not fully set in the .env file.")

engine = create_engine(f'mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

In [13]:
# --- Query tickers from raw_wikipedia_sp500 ---
with engine.connect() as conn:
    query_wiki = text("SELECT DISTINCT symbol FROM raw_wikipedia_sp500")
    df_wiki = pd.read_sql(query_wiki, conn)

In [14]:
# Standardize ticker symbols: strip any extra whitespace and convert to uppercase
df_wiki['symbol'] = df_wiki['symbol'].str.strip().str.upper()

In [15]:
# --- Query tickers already in raw_prices ---
with engine.connect() as conn:
    query_prices = text("SELECT DISTINCT symbol FROM raw_prices")
    df_prices = pd.read_sql(query_prices, conn)

if not df_prices.empty:
    processed_tickers = set(df_prices['symbol'].str.strip().str.upper())
else:
    processed_tickers = set()

all_tickers = set(df_wiki['symbol'])
new_tickers = list(all_tickers - processed_tickers)
new_tickers.sort()

In [16]:
# Define batch size (e.g., 50 tickers per execution)
batch_size = 50
tickers_to_process = new_tickers[:batch_size]

print(f"Total tickers in Wikipedia table: {len(all_tickers)}")
print(f"Tickers already processed in raw_prices: {len(processed_tickers)}")
print(f"New tickers to process in this batch: {len(tickers_to_process)}")
print("Tickers in current batch:", tickers_to_process)

Total tickers in Wikipedia table: 503
Tickers already processed in raw_prices: 335
New tickers to process in this batch: 50
Tickers in current batch: ['BF.B', 'BRK.B', 'NOC', 'NOW', 'NRG', 'NSC', 'NTAP', 'NTRS', 'NUE', 'NVDA', 'NVR', 'NWS', 'NWSA', 'NXPI', 'O', 'ODFL', 'OKE', 'OMC', 'ON', 'ORCL', 'ORLY', 'OTIS', 'OXY', 'PANW', 'PARA', 'PAYC', 'PAYX', 'PCAR', 'PCG', 'PEG', 'PEP', 'PFE', 'PFG', 'PG', 'PGR', 'PH', 'PHM', 'PKG', 'PLD', 'PLTR', 'PM', 'PNC', 'PNR', 'PNW', 'PODD', 'POOL', 'PPG', 'PPL', 'PRU', 'PSA']


In [17]:
def get_eod_prices(symbol, start="2019-01-01", end=None, resample_freq="daily"):
    """
    Fetch historical EOD price data for a given symbol from the Tiingo API.
    Returns a DataFrame with columns: date, open, high, low, close, volume, symbol.
    """
    TIINGO_KEY = os.getenv('TIINGO_KEY')
    if not TIINGO_KEY:
        raise Exception("TIINGO_KEY not set in the .env file.")
    
    base_url = f"https://api.tiingo.com/tiingo/daily/{symbol}/prices"
    params = {
        'startDate': start,
        'format': 'json',
        'token': TIINGO_KEY,
        'resampleFreq': resample_freq
    }
    if end:
        params['endDate'] = end
    
    response = requests.get(base_url, params=params, timeout=30)
    response.raise_for_status()
    data = response.json()
    if not data:
        raise Exception(f"No data returned from Tiingo for symbol: {symbol}")
    df = pd.DataFrame(data)
    columns_to_keep = ['date', 'open', 'high', 'low', 'close', 'volume']
    df = df[[col for col in columns_to_keep if col in df.columns]]
    df['date'] = pd.to_datetime(df['date'])
    for col in ['open', 'high', 'low', 'close']:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    if 'volume' in df.columns:
        df['volume'] = pd.to_numeric(df['volume'], errors='coerce')
    df['symbol'] = symbol
    df = df.sort_values('date').reset_index(drop=True)
    return df


In [18]:
for symbol in tickers_to_process:
    try:
        print(f"Processing symbol: {symbol}")
        df_symbol = get_eod_prices(symbol)
        # Wrap the insertion in a transaction so that each is rolled back if an error occurs.
        with engine.begin() as connection:
            df_symbol.to_sql(name="raw_prices", con=connection, if_exists="append", index=False)
        print(f"Symbol {symbol} processed successfully.")
    except IntegrityError as ie:
        # Duplicate entry (error code 1062) likely means data for those dates already exists.
        if "Duplicate entry" in str(ie):
            print(f"Duplicate entry error for {symbol}. Skipping insertion.")
        else:
            print(f"IntegrityError processing symbol {symbol}: {ie}")
    except OperationalError as oe:
        print(f"OperationalError processing symbol {symbol}: {oe}")
    except Exception as e:
        print(f"Error processing symbol {symbol}: {e}")
    time.sleep(1)  # Delay between requests

print("Batch processing complete. Run this cell again after one hour for the next batch.")

Processing symbol: BF.B
Error processing symbol BF.B: 404 Client Error: Not Found for url: https://api.tiingo.com/tiingo/daily/BF.B/prices?startDate=2019-01-01&format=json&token=1d49fb68fdfff0ce472e9fd2909230f678bcb647&resampleFreq=daily
Processing symbol: BRK.B
Error processing symbol BRK.B: 404 Client Error: Not Found for url: https://api.tiingo.com/tiingo/daily/BRK.B/prices?startDate=2019-01-01&format=json&token=1d49fb68fdfff0ce472e9fd2909230f678bcb647&resampleFreq=daily
Processing symbol: NOC
Symbol NOC processed successfully.
Processing symbol: NOW
Symbol NOW processed successfully.
Processing symbol: NRG
Symbol NRG processed successfully.
Processing symbol: NSC
Symbol NSC processed successfully.
Processing symbol: NTAP
Symbol NTAP processed successfully.
Processing symbol: NTRS
Symbol NTRS processed successfully.
Processing symbol: NUE
Symbol NUE processed successfully.
Processing symbol: NVDA
Symbol NVDA processed successfully.
Processing symbol: NVR
Symbol NVR processed success